In [72]:
import numpy as np
import pandas as pd
from pathlib import Path
import csv
import hvplot.pandas
import sqlalchemy as sql
import datetime as dt
from sqlalchemy import create_engine, inspect 
from tkinter import *
%matplotlib inline

In [102]:
## Function to calculate new balance for current billing period
def calc_new_balance(remaining_balance, annual_percent_rate, payment_made):
    """remaining_balance: the remaining balance on the credit card
       annual_percent_rate: the annual percentage rate
       payment_made: payment made during billing period
    """
    print(f"Card's remaining balance is ",remaining_balance)
    adjusted_balance = remaining_balance - payment_made
    monthly_percent_rate = annual_percent_rate / 12
    interest_charged = adjusted_balance * monthly_percent_rate
    new_balance = adjusted_balance + interest_charged
    print (f"Your new remaining balance is:",new_balance)
    return new_balance

In [103]:
## Read csv
ccbalance_df = pd.read_csv(Path("ccbalance.csv"))

In [104]:
display (ccbalance_df.head())
display (ccbalance_df.tail())

,card_id,card_name,remaining_balance,annual_percent_rate,payment_made
0,1,card_1,1000,0.18,100
1,2,card_2,5000,0.20,500
2,3,card_3,8000,0.14,800
3,4,card_4,2000,0.10,200
4,5,card_5,7000,0.15,700


,card_id,card_name,remaining_balance,annual_percent_rate,payment_made
0,1,card_1,1000,0.18,100
1,2,card_2,5000,0.20,500
2,3,card_3,8000,0.14,800
3,4,card_4,2000,0.10,200
4,5,card_5,7000,0.15,700


In [106]:
## Apply function to each row of dataframe
ccbalance_df["new_balance"] = ccbalance_df.apply(lambda row: calc_new_balance(row['remaining_balance'],row['annual_percent_rate'],row['payment_made']), axis = 1)
print(ccbalance_df)

Card's remaining balance is  1000
Your new remaining balance is: 913.5
Card's remaining balance is  5000
Your new remaining balance is: 4575.0
Card's remaining balance is  8000
Your new remaining balance is: 7284.0
Card's remaining balance is  2000
Your new remaining balance is: 1815.0
Card's remaining balance is  7000
Your new remaining balance is: 6378.75
   card_id card_name  remaining_balance  annual_percent_rate  payment_made  \
0        1    card_1               1000                 0.18           100   
1        2    card_2               5000                 0.20           500   
2        3    card_3               8000                 0.14           800   
3        4    card_4               2000                 0.10           200   
4        5    card_5               7000                 0.15           700   

   new_balance  
0       913.50  
1      4575.00  
2      7284.00  
3      1815.00  
4      6378.75  
